# Import Libraries and Packages

Let's go

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

In [5]:
#!unzip concrete_data_week3.zip

## Define Global Constants 

Here we will define constants that we will be using throughout the rest of the lab.

   1. We are obviously dealing with two classes, so num_classes is 2.
   2. The ResNet50 model was built and trainined using images of size (224 x 224). Therefore we will have to resize out images from (227 x 227) to (224 x 224).
   3. We will be training and validation the model using batches of 100 images.

In [6]:
num_classes = 2
image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we import from **tensorflow.keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

Next we will use the `flow_from_directory` method to get the training images as follows.

In [8]:
train_generator = data_generator.flow_from_directory (
    'concrete_data_week3/train',
    target_size = (image_resize , image_resize),
    batch_size=batch_size_training,
    class_mode='categorical'
)

Found 30000 images belonging to 2 classes.


In [9]:
validation_generator = data_generator.flow_from_directory (
    'concrete_data_week3/valid' ,
    target_size = (image_resize , image_resize),
    batch_size=batch_size_training,
    class_mode='categorical'
)

Found 10000 images belonging to 2 classes.


<br>

## Build, Compile and Fit Model

In this section we will start building our model. We will use the Sequential model from Keras.

In [10]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the outpur layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument `include_top` and set it to **False**. 

In [11]:
model.add(ResNet50 (
    include_top=False,
    pooling='avg' ,
    weights='imagenet'
))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Then we will define our output layer as a **Dense** layer that consists of two nodes and uses the **Softmax** function as the activation function.

In [12]:
model.add(Dense(num_classes, activation="softmax"))

You can access the model's layers using the *layers* attribute of our model object.

In [13]:
model.layers

You can see that our model is comprised of two sets of layers. The first set is the layers pertainig to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.

You can access the ResNet50 layers by running the following:

In [14]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense out layer. To do that, we run the following.

In [15]:
model.layers[0].trainable = False

And now using the summary attribute of the model we can see how many parameters we will need to optimize in order to train the output layer.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer

In [17]:
model.compile(optimizer='adam' , loss='categorical_crossentropy', metrics = ['accuracy', 'mse'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically that is the number of images divided by the batch size. Therefor, we define out steps per epoch as follows:

In [18]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Finally, we are ready to start training the model. Unlik a conventional deep learning trainig were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the `fit_gnerator` method. 

In [19]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
300/300 [==============================] - 1095s 4s/step - loss: 0.0379 - acc: 0.9887 - mean_squared_error: 0.0094 - val_loss: 0.0886 - val_acc: 0.9671 - val_mean_squared_error: 0.0256
Epoch 2/2
300/300 [==============================] - 1186s 4s/step - loss: 0.0097 - acc: 0.9975 - mean_squared_error: 0.0022 - val_loss: 0.0727 - val_acc: 0.9738 - val_mean_squared_error: 0.0202


Now that the model is trained, you are ready to start using it to classify images. 

Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module so go ahead and save your model.

In [20]:
model.save('classifier_resnet_model.h5')